# <font style="color:rgb(50, 120, 229);"> Transfer Learning en Keras </font>

En este cuaderno, vamos a demostrar un ejemplo de **Transfer Learning** utilizando el conjunto de datos: **Lenguaje de Señas Americano (ASL)**. 

- Instanciaremos la base convolucional de la arquitectura de red VGG-16 utilizando pesos preentrenados del conjunto de datos ImageNet. 

- Luego, agregaremos nuestro propio clasificador denso y solo entrenaremos esa parte de la red en el conjunto de datos **ASL**. 

Dado que el conjunto de datos ASL contiene tipos de imágenes (señales de manos) que no forman parte del conjunto de datos ImageNet, esperamos que el aprendizaje por transferencia sea poco efectivo.

<center>
<img src="./images/cnn_vgg_pretrained_base_ASL.webp" width="800px">
</center>

## <font style="color:rgb(50, 120, 229);"> 1. Configuración inicial </font>

In [ ]:
BATCH_SIZE = 32
LEARNING_RATE = 0.0001
EPOCHS = 50
IMG_WIDTH = 224
IMG_HEIGHT = 224

## <font style="color:rgb(50, 120, 229);"> 2. Descargar el conjunto de datos ASL </font>

In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:
!mkdir -p /root/.kaggle
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d grassknoted/asl-alphabet

In [ ]:
!unzip -q asl-alphabet.zip

## <font style="color:rgb(50, 120, 229);"> 3. Cargar el conjunto de datos ASL </font>

El conjunto de datos ASL tiene la siguiente estructura de directorios:

```bash
dataset_ASL_150/
    |______ A/
    |______ B/
    |______ C/
    |______ D/
    |______ E/
    |______ F/
    |______ G/
    |______ H/
    |______ I/
    |______ J/
    |______ K/
    |______ L/
    |______ M/
    |______ N/
    |______ O/
    |______ P/
    |______ Q/
    |______ R/
    |______ S/
    |______ T/
    |______ U/
    |______ V/
    |______ W/
    |______ X/
    |______ Y/
    |______ Z/
    |______ del/
    |______ nothing/
    |______ space/

**Este dataset no tiene un conjunto de datos de validación.**

Por lo tanto, dividiremos el conjunto de datos en un conjunto de entrenamiento y un conjunto de validación especificando los parámetros de la función `image_dataset_from_directory` de TensorFlow de la siguiente manera:

In [ ]:
from keras.utils import image_dataset_from_directory 

train_dataset = image_dataset_from_directory(
    "./dataset_ASL_150/",
    labels="inferred",
    label_mode="categorical",
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    validation_split=0.2,
    subset="training",
    shuffle=True,
    seed=42
)

In [ ]:
#TODO: Crea un conjunto de validación con el 20% de los datos especificando el subset como "validation"

In [ ]:
from matplotlib import pyplot as plt

class_names = train_dataset.class_names

plt.figure(figsize=(10, 10))

for images, labels in train_dataset.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        label = labels[i]
        class_name = label.numpy().argmax()
        plt.title(class_name)
        plt.axis("off")

plt.show()

## <font style="color:rgb(50, 120, 229);"> 4. Crea el modelo </font>

En esta parte del modelo, vamos a especificar una capa de entrada y la base convolucional de la red VGG-16. (Puedes incluir Data Augmentation después de la capa de entrada si lo deseas).

Por último, agregaremos un clasificador denso y compilaremos el modelo.

**model.trainable = False** significa que solo entrenaremos el clasificador denso y no la base convolucional de la red VGG-16. Esto se conoce como **Transfer Learning**.

In [ ]:
from keras.applications import VGG16

base_model = VGG16(
    include_top=False, #No incluir la capa densa
    weights='imagenet',
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
)

#Congelar las capas del modelo base
base_model.trainable = False

In [ ]:
from keras.layers import Input, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import preprocess_input

input_layer = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
x = preprocess_input(input_layer) # Agregar preprocesamiento de VGG16

x = base_model(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
output = Dense(29, activation='softmax')(x)

model = Model(input_layer, output)

model.summary()

## <font style="color:rgb(50, 120, 229);"> 5. Entrenar el modelo </font>

In [ ]:
from keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True
)

In [ ]:
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=validation_dataset,
    callbacks=[early_stopping]
)

## <font style="color:rgb(50, 120, 229);"> 6. Visualizar los resultados </font>

In [ ]:
plt.figure(figsize=(20, 10))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

## <font style="color:rgb(50, 120, 229);"> 7. Visualizar algunas predicciones </font>

In [ ]:
plt.figure(figsize=(20, 10))

for images, labels in validation_dataset.take(1):
    predictions = model.predict(images)
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        
        label = labels[i]
        class_name = label.numpy().argmax()
        plt.title(f"Real: {class_names[class_name]} - Predicción: {class_names[predictions[i].argmax()]}")
        plt.axis("off")

        plt.imshow(images[i].numpy().astype("uint8"))

plt.show()


## <font style="color:rgb(50, 120, 229);"> 8. Conclusiones </font>

En este cuaderno, usamos un conjunto de datos diferente para experimentar con **Transfer Learning**, y vimos que la precisión de validación en el conjunto de datos **ASL** no fue tan buena. 

La razón de esto es que el aprendizaje por transferencia se basó en el conjunto de datos ImageNet, que no tiene representaciones para señales de manos. Por lo tanto, la base convolucional que se utilizó para el aprendizaje por transferencia, en este caso, carece de las características de nivel inferior que serían útiles para clasificar señales de manos.
